In [ ]:
import pandas as pd

novy_df = pd.read_csv('finaldataset_regresni_analyza.csv')

print(novy_df.head())

overeniCNO_df = novy_df[['spoofing', 'time', 'g_avg_cn0']]

print(overeniCNO_df.head())

print(f"DataFrame má {overeniCNO_df.shape[0]} řádků a {overeniCNO_df.shape[1]} sloupců.")



In [ ]:
# TEST NORMALITY
# To ensure that the T-test is valid, we need to check whether the data follows a normal distribution.
# The Shapiro-Wilk test is suitable for small datasets, but for larger datasets, this test may be unsuitable.

from scipy import stats

# Splitting the data into two parts: one with spoofing detected and the other without spoofing detected

cn0_spoofing = overeniCNO_df[overeniCNO_df['spoofing'] == True]['g_avg_cn0']
cn0_no_spoofing = overeniCNO_df[overeniCNO_df['spoofing'] == False]['g_avg_cn0']

# Test normality for Cn0 - spoofing
shapiro_spoofing = stats.shapiro(cn0_spoofing)
shapiro_no_spoofing = stats.shapiro(cn0_no_spoofing)

print(f'Shapiro-Wilk test pro spoofing: Statistika = {shapiro_spoofing.statistic}, P-hodnota = {shapiro_spoofing.pvalue}')
print(f'Shapiro-Wilk test pro nespoofing: Statistika = {shapiro_no_spoofing.statistic}, P-hodnota = {shapiro_no_spoofing.pvalue}')


In [ ]:
# # TEST NORMALITY
# #Kolmogorov-Smirnov test:
# from scipy.stats import kstest

# # Kolmogorov-Smirnov test - spoofing
# ks_test_spoofing = kstest(cn0_spoofing, 'norm')
# print(f'Kolmogorov-Smirnov test pro spoofing: Statistika = {ks_test_spoofing.statistic}, P-hodnota = {ks_test_spoofing.pvalue}')

# # Kolmogorov-Smirnov test - clear
# ks_test_no_spoofing = kstest(cn0_no_spoofing, 'norm')
# print(f'Kolmogorov-Smirnov test pro nespoofing: Statistika = {ks_test_no_spoofing.statistic}, P-hodnota = {ks_test_no_spoofing.pvalue}')

# # Test Statistic:
# # Test statistic = 0.5 for spoofing and 0.904 for non-spoofing: A higher test statistic value indicates that the data are further from a normal distribution. Therefore, in both cases, the data deviate significantly from a normal distribution.
# # P-value:
# # P-value = 3.775336366111096e-269 for spoofing and 0.0 for non-spoofing: Very small p-values (including 0.0) indicate that it is extremely unlikely the data follow a normal distribution. In both cases, the data are significantly non-normal.


Kolmogorov-Smirnov test pro spoofing: Statistika = 0.5, P-hodnota = 3.775336366111096e-269
Kolmogorov-Smirnov test pro nespoofing: Statistika = 0.904031081604096, P-hodnota = 0.0


In [ ]:
# NON-PARAMETRIC TEST
# For data that do not follow a normal distribution, as is the case here (as discussed above), we use non-parametric tests.
# In our case, we will use the Mann-Whitney U test, which is a non-parametric test used to compare two independent groups.
# It is the non-parametric equivalent of the T-test, which does not require normal distribution, but it is statistically "weaker" (i.e., it does not carry the same evidentiary weight or precision).
# from scipy import stats

# Mann-Whitney U test for CN0 during spoofing and non-spoofing

mann_whitney_result = stats.mannwhitneyu(cn0_spoofing, cn0_no_spoofing)
print(f'Mann-Whitney U test: Statistika = {mann_whitney_result.statistic}, P-hodnota = {mann_whitney_result.pvalue}')

# The statistic = 5,376,184.5: The numerical result of the Mann-Whitney U test is used to calculate the p-value.
# P-value = 0.0: A practically zero p-value means that the difference between CN0 values during spoofing and non-spoofing is statistically significant. The CN0 values are significantly different between the cases where spoofing was detected and the cases where it was not.

Mann-Whitney U test: Statistika = 5376184.5, P-hodnota = 0.0


In [ ]:
# The Kolmogorov-Smirnov test confirmed that the data are not normally distributed.
# The Mann-Whitney U test showed that the difference between CN0 values during spoofing and non-spoofing is statistically significant.
# This significant difference suggests that spoofing affects CN0 values, confirming the importance of CN0 for GPS spoofing detection.
# Therefore, we can accept the hypothesis that spoofing has an impact on signal quality (measured by CN0).

In [ ]:
# STATISTICAL ANALYSIS OF CN0 VALUES
# Calculation of statistics: mean, median, and variance for both groups to determine if there are differences between them.

print(f'Průměr CN0 při spoofingu: {cn0_spoofing.mean()}')
print(f'Průměr CN0 mimo spoofing: {cn0_no_spoofing.mean()}')

print(f'Medián CN0 při spoofingu: {cn0_spoofing.median()}')
print(f'Medián CN0 mimo spoofing: {cn0_no_spoofing.median()}')

print(f'Rozptyl CN0 při spoofingu: {cn0_spoofing.var()}')
print(f'Rozptyl CN0 mimo spoofing: {cn0_no_spoofing.var()}')

# Spoofing has a significant impact on lowering the CN0 value, the signal is very weak or non-existent during spoofing.
# A significantly lower average and median CN0 during spoofing provides a clear indicator that spoofing affects the quality of the GPS signal and can be used as a metric for spoofing detection.

In [ ]:
# T-test to determine whether the means of CN0 during spoofing and non-spoofing are statistically different. Generally used for comparing the means of two independent groups. 

t_stat, p_value = stats.ttest_ind(cn0_spoofing, cn0_no_spoofing)
print(f'T-statistika: {t_stat}')
print(f'P-hodnota: {p_value}')

# The t-statistic shows the size of the difference between the means of two groups (CN0 during spoofing and non-spoofing) compared to the variance of these values.
# A value of -90.21 means that the difference between the means is very significant, with the negative value indicating that CN0 during spoofing is significantly lower than CN0 outside of spoofing.
# The p-value of 0.0 (often written as < 0.001) means it is extremely unlikely that the difference between CN0 during and outside of spoofing is due to chance.
# The t-statistic of -90.21 and the p-value of 0.0 clearly indicate that the difference in CN0 values between spoofing and non-spoofing is not random and that spoofing has a significant impact on lowering the CN0 value.
# This result supports the hypothesis that during spoofing, there is a significant reduction in signal strength, which could be a key indicator for spoofing detection.